# 教師なし学習: 異常検知（Anomaly Detection）

このノートブックでは、タイタニックデータセットを使って**異常検知（Anomaly Detection）**の手法を学びます。

## 異常検知とは？

**異常検知**は、正常なデータパターンから外れた「異常値」や「外れ値」を検出する教師なし学習の手法です。

### 実務での応用例

1. **不正検知**: クレジットカード不正利用、マネーロンダリング
2. **システム監視**: サーバーの異常動作、ネットワーク侵入検知
3. **製造業**: 不良品検出、機器の故障予兆
4. **医療**: 異常な検査値、珍しい疾患パターン
5. **データクレンジング**: データの品質チェック、入力ミス検出

### タイタニックデータでの異常検知の目的

- 通常とは異なる特徴を持つ乗客を発見する
- データの品質をチェックする（入力ミス、外れ値など）
- 異常値を除外・処理して予測モデルの精度を向上させる

## 学習する手法

1. **Isolation Forest**: ランダムに分割して異常を隔離
2. **One-Class SVM**: サポートベクターマシンで正常領域を学習
3. **Local Outlier Factor (LOF)**: 局所密度から異常を判定

## 1. ライブラリのインポート

In [ ]:
# データ処理
import pandas as pd
import numpy as np

# 可視化
import matplotlib.pyplot as plt
import seaborn as sns

# 機械学習（異常検知）
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor

# 前処理・評価
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer

# 設定
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
pd.set_option('display.max_columns', None)
np.random.seed(42)

%matplotlib inline

## 2. データの読み込み

In [ ]:
# データの読み込み
train = pd.read_csv('../data/raw/train.csv')
test = pd.read_csv('../data/raw/test.csv')

print(f'訓練データサイズ: {train.shape}')
print(f'テストデータサイズ: {test.shape}')

# 最初の数行を確認
train.head()

## 3. データの前処理

異常検知を行うために、データを数値化して前処理します。

### 前処理の手順

1. **必要な特徴量を選択**
2. **カテゴリ変数を数値化**（Label Encoding）
3. **欠損値を補完**
4. **特徴量を標準化**（平均0、分散1に変換）

In [ ]:
# データのコピーを作成（元データを保持）
df = train.copy()

# 特徴量エンジニアリング
# 家族サイズ
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1

# 単身かどうか
df['IsAlone'] = (df['FamilySize'] == 1).astype(int)

# 敬称（Title）を抽出
df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

# 敬称を整理
title_mapping = {
    'Mr': 'Mr', 'Miss': 'Miss', 'Mrs': 'Mrs', 'Master': 'Master',
    'Dr': 'Rare', 'Rev': 'Rare', 'Col': 'Rare', 'Major': 'Rare',
    'Mlle': 'Miss', 'Countess': 'Rare', 'Ms': 'Miss', 'Lady': 'Rare',
    'Jonkheer': 'Rare', 'Don': 'Rare', 'Dona': 'Rare', 'Mme': 'Mrs',
    'Capt': 'Rare', 'Sir': 'Rare'
}
df['Title'] = df['Title'].map(title_mapping)

# 使用する特徴量を選択
features = ['Pclass', 'Sex', 'Age', 'Fare', 'Embarked', 'FamilySize', 'IsAlone', 'Title']

# 特徴量のみを抽出
X = df[features].copy()

print("前処理前のデータ:")
print(X.head())
print(f"\n欠損値の数:\n{X.isnull().sum()}")

In [ ]:
# カテゴリ変数を数値化
categorical_features = ['Sex', 'Embarked', 'Title']

for col in categorical_features:
    # 欠損値を最頻値で補完してからエンコード
    X[col].fillna(X[col].mode()[0], inplace=True)
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])

print("カテゴリ変数エンコード後:")
print(X.head())

In [ ]:
# 数値変数の欠損値を中央値で補完
imputer = SimpleImputer(strategy='median')
X_imputed = imputer.fit_transform(X)

# DataFrame形式に戻す
X_processed = pd.DataFrame(X_imputed, columns=X.columns)

print("欠損値補完後:")
print(f"欠損値の数: {X_processed.isnull().sum().sum()}")
print(X_processed.head())

In [ ]:
# 標準化（平均0、分散1に変換）
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_processed)

# DataFrame形式に戻す
X_scaled_df = pd.DataFrame(X_scaled, columns=X_processed.columns)

print("標準化後のデータ:")
print(X_scaled_df.head())
print(f"\n平均: {X_scaled_df.mean().round(2)}")
print(f"\n標準偏差: {X_scaled_df.std().round(2)}")

## 4. 手法1: Isolation Forest

### Isolation Forestとは？

**Isolation Forest**は、異常値を「隔離しやすい」という性質を利用した手法です。

#### 基本的な考え方

1. **正常データ**: 密集しているため、ランダムに分割すると隔離するのに多くの分割が必要
2. **異常データ**: 離れているため、少ない分割で隔離できる

#### 仕組み

1. ランダムに特徴量を選択
2. ランダムに分割点を選んでデータを分割
3. これを繰り返して木構造（ツリー）を作成
4. **分割回数が少ないデータ = 異常**と判定

#### 主なパラメータ

- **n_estimators**: 作成する木の数（デフォルト: 100）
- **contamination**: 異常データの割合の推定値（デフォルト: 0.1 = 10%）
- **max_samples**: 各木で使用するサンプル数
- **random_state**: 乱数シード（再現性のため）

In [ ]:
# Isolation Forestモデルの作成
# contamination: 異常と判定するデータの割合（5%と仮定）
iso_forest = IsolationForest(
    n_estimators=100,
    contamination=0.05,
    random_state=42,
    n_jobs=-1  # 全CPUコアを使用
)

# 学習と予測
# 予測結果: 1 = 正常, -1 = 異常
iso_predictions = iso_forest.fit_predict(X_scaled)

# 異常スコアを計算（値が小さいほど異常）
iso_scores = iso_forest.score_samples(X_scaled)

# 結果を元のDataFrameに追加
df['iso_prediction'] = iso_predictions
df['iso_score'] = iso_scores

print(f"異常と判定されたデータ数: {(iso_predictions == -1).sum()}件")
print(f"正常と判定されたデータ数: {(iso_predictions == 1).sum()}件")
print(f"異常データの割合: {(iso_predictions == -1).sum() / len(iso_predictions) * 100:.2f}%")

### Isolation Forestの結果を可視化

In [ ]:
# 異常スコアの分布を可視化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ヒストグラム
axes[0].hist(df['iso_score'], bins=50, alpha=0.7, color='skyblue', edgecolor='black')
axes[0].axvline(df[df['iso_prediction'] == -1]['iso_score'].max(), 
                color='red', linestyle='--', label='異常の閾値')
axes[0].set_xlabel('Anomaly Score')
axes[0].set_ylabel('Count')
axes[0].set_title('Distribution of Isolation Forest Anomaly Scores')
axes[0].legend()

# ボックスプロット（正常 vs 異常）
df_plot = df.copy()
df_plot['Label'] = df_plot['iso_prediction'].map({1: 'Normal', -1: 'Anomaly'})
sns.boxplot(data=df_plot, x='Label', y='iso_score', ax=axes[1])
axes[1].set_title('Anomaly Score: Normal vs Anomaly')
axes[1].set_ylabel('Anomaly Score')

plt.tight_layout()
plt.show()

In [ ]:
# PCAで2次元に削減して可視化
pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_scaled)

# 可視化
plt.figure(figsize=(10, 7))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], 
                     c=iso_predictions, 
                     cmap='coolwarm', 
                     alpha=0.6,
                     edgecolors='black',
                     linewidths=0.5)
plt.colorbar(scatter, label='Prediction (1=Normal, -1=Anomaly)')
plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)')
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)')
plt.title('Isolation Forest: Normal vs Anomaly (PCA Visualization)')
plt.grid(True, alpha=0.3)
plt.show()

print(f"PCAによる説明分散比率: PC1={pca.explained_variance_ratio_[0]*100:.2f}%, PC2={pca.explained_variance_ratio_[1]*100:.2f}%")

### 異常と判定されたデータを分析

In [ ]:
# 異常データを抽出
anomalies_iso = df[df['iso_prediction'] == -1].copy()

print("異常と判定されたデータ（上位10件）:")
print(anomalies_iso[['PassengerId', 'Name', 'Pclass', 'Sex', 'Age', 'Fare', 
                      'FamilySize', 'Survived', 'iso_score']].head(10))

In [ ]:
# 正常データと異常データの特徴を比較
normal_iso = df[df['iso_prediction'] == 1]

comparison = pd.DataFrame({
    'Normal_Mean': normal_iso[['Pclass', 'Age', 'Fare', 'FamilySize', 'Survived']].mean(),
    'Anomaly_Mean': anomalies_iso[['Pclass', 'Age', 'Fare', 'FamilySize', 'Survived']].mean()
})
comparison['Difference'] = comparison['Anomaly_Mean'] - comparison['Normal_Mean']

print("正常データと異常データの平均値比較:")
print(comparison.round(2))

In [ ]:
# 特徴量ごとの分布比較
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

features_to_plot = ['Age', 'Fare', 'FamilySize', 'Pclass']

for idx, feature in enumerate(features_to_plot):
    ax = axes[idx // 2, idx % 2]
    
    ax.hist([normal_iso[feature].dropna(), anomalies_iso[feature].dropna()], 
            bins=30, alpha=0.7, label=['Normal', 'Anomaly'])
    ax.set_xlabel(feature)
    ax.set_ylabel('Count')
    ax.set_title(f'{feature} Distribution: Normal vs Anomaly')
    ax.legend()

plt.tight_layout()
plt.show()

## 5. 手法2: One-Class SVM

### One-Class SVMとは？

**One-Class SVM**は、正常データの領域を学習し、その領域から外れたデータを異常と判定する手法です。

#### 基本的な考え方

1. 正常データを囲む**決定境界**を学習
2. この境界の外側にあるデータを異常と判定
3. カーネル関数を使って非線形な境界も学習可能

#### 主なパラメータ

- **nu**: 異常データの上限割合（0 < nu ≤ 1）。小さいほど厳しく判定
- **kernel**: カーネル関数（'rbf', 'linear', 'poly'など）
  - 'rbf': ガウシアンカーネル（デフォルト、非線形）
  - 'linear': 線形カーネル
- **gamma**: RBFカーネルのパラメータ（'scale'または数値）

#### Isolation Forestとの違い

| 項目 | Isolation Forest | One-Class SVM |
|------|------------------|---------------|
| アプローチ | 異常を隔離しやすいか | 正常領域を学習 |
| 計算速度 | 速い | 遅い（大規模データでは注意） |
| 決定境界 | 明示的でない | 明示的 |
| 高次元データ | 得意 | やや苦手 |

In [ ]:
# One-Class SVMモデルの作成
ocsvm = OneClassSVM(
    kernel='rbf',
    gamma='scale',
    nu=0.05  # 異常データの割合を5%と仮定
)

# 学習と予測
# 予測結果: 1 = 正常, -1 = 異常
ocsvm_predictions = ocsvm.fit_predict(X_scaled)

# 異常スコアを計算（決定関数の値、正なら正常、負なら異常）
ocsvm_scores = ocsvm.decision_function(X_scaled)

# 結果を元のDataFrameに追加
df['ocsvm_prediction'] = ocsvm_predictions
df['ocsvm_score'] = ocsvm_scores

print(f"異常と判定されたデータ数: {(ocsvm_predictions == -1).sum()}件")
print(f"正常と判定されたデータ数: {(ocsvm_predictions == 1).sum()}件")
print(f"異常データの割合: {(ocsvm_predictions == -1).sum() / len(ocsvm_predictions) * 100:.2f}%")

### One-Class SVMの結果を可視化

In [ ]:
# 異常スコアの分布を可視化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ヒストグラム
axes[0].hist(df['ocsvm_score'], bins=50, alpha=0.7, color='lightcoral', edgecolor='black')
axes[0].axvline(0, color='red', linestyle='--', label='決定境界（0）')
axes[0].set_xlabel('Decision Score')
axes[0].set_ylabel('Count')
axes[0].set_title('Distribution of One-Class SVM Decision Scores')
axes[0].legend()

# ボックスプロット（正常 vs 異常）
df_plot = df.copy()
df_plot['Label'] = df_plot['ocsvm_prediction'].map({1: 'Normal', -1: 'Anomaly'})
sns.boxplot(data=df_plot, x='Label', y='ocsvm_score', ax=axes[1])
axes[1].axhline(0, color='red', linestyle='--', alpha=0.7)
axes[1].set_title('Decision Score: Normal vs Anomaly')
axes[1].set_ylabel('Decision Score')

plt.tight_layout()
plt.show()

In [ ]:
# PCAで2次元に削減して可視化
plt.figure(figsize=(10, 7))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], 
                     c=ocsvm_predictions, 
                     cmap='coolwarm', 
                     alpha=0.6,
                     edgecolors='black',
                     linewidths=0.5)
plt.colorbar(scatter, label='Prediction (1=Normal, -1=Anomaly)')
plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)')
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)')
plt.title('One-Class SVM: Normal vs Anomaly (PCA Visualization)')
plt.grid(True, alpha=0.3)
plt.show()

### 異常と判定されたデータを分析

In [ ]:
# 異常データを抽出
anomalies_ocsvm = df[df['ocsvm_prediction'] == -1].copy()

print("異常と判定されたデータ（上位10件）:")
print(anomalies_ocsvm[['PassengerId', 'Name', 'Pclass', 'Sex', 'Age', 'Fare', 
                        'FamilySize', 'Survived', 'ocsvm_score']].head(10))

## 6. 手法3: Local Outlier Factor (LOF)

### Local Outlier Factorとは？

**LOF**は、各データポイントの**局所密度**を計算し、周囲と比較して密度が低いデータを異常と判定する手法です。

#### 基本的な考え方

1. 各データポイントについて、k近傍のデータを見つける
2. その近傍の密度を計算
3. 近傍の密度と比較して、自分の密度が低ければ異常

#### 主なパラメータ

- **n_neighbors**: 近傍の数（デフォルト: 20）
- **contamination**: 異常データの割合の推定値

#### 他の手法との違い

- **Isolation Forest**: グローバルな異常検知
- **One-Class SVM**: 正常領域の境界を学習
- **LOF**: **局所的な密度**に基づく異常検知
  - クラスタが複数ある場合に有効
  - 密度が異なる領域でも検出可能

In [ ]:
# LOFモデルの作成
lof = LocalOutlierFactor(
    n_neighbors=20,
    contamination=0.05,
    n_jobs=-1
)

# 学習と予測
# 予測結果: 1 = 正常, -1 = 異常
lof_predictions = lof.fit_predict(X_scaled)

# 異常スコア（負の外れ値因子）
# 値が小さい（より負）ほど異常
lof_scores = lof.negative_outlier_factor_

# 結果を元のDataFrameに追加
df['lof_prediction'] = lof_predictions
df['lof_score'] = lof_scores

print(f"異常と判定されたデータ数: {(lof_predictions == -1).sum()}件")
print(f"正常と判定されたデータ数: {(lof_predictions == 1).sum()}件")
print(f"異常データの割合: {(lof_predictions == -1).sum() / len(lof_predictions) * 100:.2f}%")

### LOFの結果を可視化

In [ ]:
# 異常スコアの分布を可視化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ヒストグラム
axes[0].hist(df['lof_score'], bins=50, alpha=0.7, color='lightgreen', edgecolor='black')
axes[0].axvline(df[df['lof_prediction'] == -1]['lof_score'].max(), 
                color='red', linestyle='--', label='異常の閾値')
axes[0].set_xlabel('LOF Score (Negative Outlier Factor)')
axes[0].set_ylabel('Count')
axes[0].set_title('Distribution of LOF Scores')
axes[0].legend()

# ボックスプロット（正常 vs 異常）
df_plot = df.copy()
df_plot['Label'] = df_plot['lof_prediction'].map({1: 'Normal', -1: 'Anomaly'})
sns.boxplot(data=df_plot, x='Label', y='lof_score', ax=axes[1])
axes[1].set_title('LOF Score: Normal vs Anomaly')
axes[1].set_ylabel('LOF Score')

plt.tight_layout()
plt.show()

In [ ]:
# PCAで2次元に削減して可視化
plt.figure(figsize=(10, 7))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], 
                     c=lof_predictions, 
                     cmap='coolwarm', 
                     alpha=0.6,
                     edgecolors='black',
                     linewidths=0.5)
plt.colorbar(scatter, label='Prediction (1=Normal, -1=Anomaly)')
plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)')
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)')
plt.title('LOF: Normal vs Anomaly (PCA Visualization)')
plt.grid(True, alpha=0.3)
plt.show()

## 7. 手法の比較

3つの異常検知手法（Isolation Forest、One-Class SVM、LOF）の結果を比較します。

In [ ]:
# 各手法で検出された異常データの重複を確認
from matplotlib_venn import venn3

# 異常と判定されたインデックスを取得
iso_anomalies = set(df[df['iso_prediction'] == -1].index)
ocsvm_anomalies = set(df[df['ocsvm_prediction'] == -1].index)
lof_anomalies = set(df[df['lof_prediction'] == -1].index)

# ベン図で可視化
plt.figure(figsize=(10, 7))
venn3([iso_anomalies, ocsvm_anomalies, lof_anomalies], 
      set_labels=('Isolation Forest', 'One-Class SVM', 'LOF'))
plt.title('Overlap of Anomalies Detected by Different Methods')
plt.show()

# 統計情報を表示
print("各手法で検出された異常データ数:")
print(f"Isolation Forest: {len(iso_anomalies)}件")
print(f"One-Class SVM: {len(ocsvm_anomalies)}件")
print(f"LOF: {len(lof_anomalies)}件")
print(f"\n全ての手法で異常と判定: {len(iso_anomalies & ocsvm_anomalies & lof_anomalies)}件")
print(f"少なくとも1つの手法で異常と判定: {len(iso_anomalies | ocsvm_anomalies | lof_anomalies)}件")

In [ ]:
# 3つの手法すべてで異常と判定されたデータを確認
all_methods_anomalies = iso_anomalies & ocsvm_anomalies & lof_anomalies

if len(all_methods_anomalies) > 0:
    print("全ての手法で異常と判定されたデータ:")
    print(df.loc[list(all_methods_anomalies), 
                 ['PassengerId', 'Name', 'Pclass', 'Sex', 'Age', 'Fare', 
                  'FamilySize', 'Survived']])
else:
    print("全ての手法で異常と判定されたデータはありません。")

In [ ]:
# 各手法の異常スコアの相関を確認
score_correlation = df[['iso_score', 'ocsvm_score', 'lof_score']].corr()

plt.figure(figsize=(8, 6))
sns.heatmap(score_correlation, annot=True, fmt='.2f', cmap='coolwarm', center=0)
plt.title('Correlation of Anomaly Scores')
plt.show()

print("異常スコアの相関:")
print(score_correlation)

### 手法の比較まとめ

| 手法 | メリット | デメリット | 適用場面 |
|------|---------|-----------|----------|
| **Isolation Forest** | ・高速<br>・高次元データに強い<br>・パラメータ調整が簡単 | ・決定境界が不明確<br>・局所的なパターンを見逃す可能性 | ・大規模データ<br>・高次元データ<br>・グローバルな異常検知 |
| **One-Class SVM** | ・決定境界が明確<br>・非線形境界を学習可能<br>・理論的基盤が強固 | ・計算コストが高い<br>・パラメータ調整が難しい<br>・大規模データには不向き | ・小〜中規模データ<br>・明確な境界が欲しい場合<br>・正常領域が明確 |
| **LOF** | ・局所的なパターンを検出<br>・密度が異なる領域でも検出可能<br>・クラスタごとの異常検知 | ・計算コストが高い<br>・近傍数の選択が重要<br>・大規模データには不向き | ・複数クラスタがある<br>・密度が不均一<br>・局所的な異常検知 |

## 8. 応用: 生存予測への活用

異常検知の結果を使って、以下の2つのアプローチを試します：

1. **異常値を除外**してモデルを学習
2. **異常スコアを特徴量として追加**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# 生存予測用のデータを準備
y = df['Survived'].values

# ベースラインモデル（全データを使用）
rf_baseline = RandomForestClassifier(n_estimators=100, random_state=42)
baseline_scores = cross_val_score(rf_baseline, X_scaled, y, cv=5, scoring='accuracy')

print("ベースラインモデル（全データ）:")
print(f"平均精度: {baseline_scores.mean():.4f} (+/- {baseline_scores.std():.4f})")

In [ ]:
# アプローチ1: 異常値を除外してモデルを学習
# Isolation Forestで異常と判定されたデータを除外
normal_mask = df['iso_prediction'] == 1
X_normal = X_scaled[normal_mask]
y_normal = y[normal_mask]

rf_filtered = RandomForestClassifier(n_estimators=100, random_state=42)
filtered_scores = cross_val_score(rf_filtered, X_normal, y_normal, cv=5, scoring='accuracy')

print("\n異常値除外モデル:")
print(f"平均精度: {filtered_scores.mean():.4f} (+/- {filtered_scores.std():.4f})")
print(f"訓練データ数: {len(X_normal)}件（{len(X_scaled) - len(X_normal)}件を除外）")

In [ ]:
# アプローチ2: 異常スコアを特徴量として追加
X_with_scores = np.column_stack([
    X_scaled,
    df['iso_score'].values.reshape(-1, 1),
    df['ocsvm_score'].values.reshape(-1, 1),
    df['lof_score'].values.reshape(-1, 1)
])

rf_with_scores = RandomForestClassifier(n_estimators=100, random_state=42)
scores_added = cross_val_score(rf_with_scores, X_with_scores, y, cv=5, scoring='accuracy')

print("\n異常スコア追加モデル:")
print(f"平均精度: {scores_added.mean():.4f} (+/- {scores_added.std():.4f})")

In [ ]:
# 結果を比較
results = pd.DataFrame({
    'Model': ['Baseline', 'Filtered (No Anomalies)', 'With Anomaly Scores'],
    'Mean Accuracy': [baseline_scores.mean(), filtered_scores.mean(), scores_added.mean()],
    'Std Dev': [baseline_scores.std(), filtered_scores.std(), scores_added.std()]
})

print("\n結果の比較:")
print(results.round(4))

# 可視化
plt.figure(figsize=(10, 6))
x_pos = np.arange(len(results))
plt.bar(x_pos, results['Mean Accuracy'], yerr=results['Std Dev'], 
        alpha=0.7, capsize=10, color=['skyblue', 'lightcoral', 'lightgreen'])
plt.xlabel('Model')
plt.ylabel('Accuracy')
plt.title('Comparison of Models with Different Anomaly Handling')
plt.xticks(x_pos, results['Model'], rotation=15, ha='right')
plt.ylim(0.7, 0.85)
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

## 9. まとめ

### 学んだこと

#### 1. 異常検知の基礎
- 異常検知は正常データから外れたパターンを見つける教師なし学習
- 不正検知、システム監視、品質管理など幅広い応用がある

#### 2. 3つの主要な手法

**Isolation Forest**
- ランダム分割で異常を隔離
- 高速で高次元データに強い
- まず最初に試すべき手法

**One-Class SVM**
- 正常領域の境界を学習
- 決定境界が明確
- 小〜中規模データに適している

**Local Outlier Factor (LOF)**
- 局所密度に基づく検出
- 複数クラスタや密度が不均一な場合に有効
- 局所的な異常パターンを検出

#### 3. 実践的なポイント

- **複数の手法を試す**: 1つの手法だけでなく、複数を比較することが重要
- **ドメイン知識を活用**: 検出された異常が本当に異常か、人間の目で確認
- **contamination（異常の割合）の設定**: データに応じて適切に設定
- **可視化が重要**: PCAなどで次元削減して結果を確認

#### 4. 予測タスクへの応用

- 異常値を除外することで、場合によってはモデルの精度が向上
- 異常スコアを特徴量として追加する方法も有効
- ただし、除外しすぎると過学習のリスクもあるので注意

### 次のステップ

1. **他の異常検知手法を学ぶ**
   - Autoencoder（ディープラーニング）
   - DBSCAN（クラスタリングベース）
   - Elliptic Envelope（統計的手法）

2. **他のデータセットで実践**
   - クレジットカード不正検知データ
   - ネットワーク侵入検知データ
   - センサーデータの異常検知

3. **パラメータチューニング**
   - Grid Searchで最適なパラメータを探索
   - 異常の割合（contamination）の影響を調査

4. **評価指標を学ぶ**
   - 正解ラベルがある場合の評価（Precision, Recall, F1-score）
   - 正解ラベルがない場合の評価（Silhouette Score、可視化）

### 参考資料

- [scikit-learn: Outlier Detection](https://scikit-learn.org/stable/modules/outlier_detection.html)
- [Isolation Forest論文](https://cs.nju.edu.cn/zhouzh/zhouzh.files/publication/icdm08b.pdf)
- [LOF論文](https://www.dbs.ifi.lmu.de/Publikationen/Papers/LOF.pdf)